In [1]:
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
from helper import util_preprocessing, util_pids

import math

# 1. Get train and test split per category

## 1.1 One hot encoded

### 1.1.1 Subclasses

In [2]:
# read one hot encoded data
subclasses_one_hot = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/subclasses_one_hot_encoded.csv', 
                                 sep=";", index_col=0)
subclasses_one_hot

,subClass,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,10,decade,2,1,2,108191420,['p3a'],0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,abbess,nun,12,7,11,403873869,"['p4p8ap3a', 'p3a', 'p8b', 'p4', 'p8b', 'p27a'...",1,0,0,...,1,0,0,0,0,0,1,1,0,0
2,abbot,person,48,7,32,156276663,"['p4p1', 'p1', 'p1', 'p1', 'p1', 'p8b', 'p3a',...",1,0,0,...,1,0,0,1,0,0,1,1,0,0
3,actuary,statistician,3,2,3,512259817,['p4p8a'],0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,activist,person,172,16,101,485263422,"['p4p23dp2p39p1p23ap23bp8ap3ap43', 'p6p8bp3ap4...",1,0,0,...,1,0,1,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793,zimbabweans,person,1,1,1,144540438,['p1'],1,0,0,...,0,0,0,0,0,0,0,0,0,0
3794,zine,work,32,10,26,170394103,"['p5p1p20a', 'p8a', 'p8a', 'p8a', 'p3a', 'p3a'...",1,0,0,...,0,0,0,1,0,0,1,1,0,0
3795,zine,periodical,23,5,15,189735779,"['p4p1', 'p1p3a', 'p3a', 'p12a', 'p3a', 'p1', ...",1,0,0,...,1,0,0,0,0,0,1,0,0,0
3796,zine,publication,124,12,96,418332088,"['p5p34p4p1p8ap28dp3a', 'p1p8a', 'p1', 'p1', '...",1,0,0,...,1,0,0,1,0,0,1,0,0,0


In [3]:
# drop potential duplicates in subclasses
subclasses_one_hot.drop_duplicates(subset=['id'], keep='first',
                                  inplace=True)
subclasses_one_hot.shape

(3798, 64)

### 1.1.2 Types

In [4]:
# read one hot encoded types
types_one_hot = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/types_one_hot_encoded.csv', 
                            sep=";", index_col=0)

types_one_hot

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,1988,album,4,2,4,89520215,"['p3a', 'p5', 'p5', 'p3a']",0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,196,year,1,1,1,250210719,['p8b'],0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,53,year,2,1,1,333976287,['p8b'],0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2007,year,104,6,57,179643001,"['p25p8ap8b', 'p8b', 'p8b', 'p25', 'p8b', 'p8b...",1,0,0,...,0,0,0,0,0,0,1,1,0,0
4,2,aircraft,21,4,16,472381603,"['p8b', 'p8b', 'p8b', 'p3a', 'p8b', 'p5', 'p8b...",0,0,0,...,0,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118162,india,city,251,29,206,205155758,"['p5p4p1p15bp23ap24p8ap3a', 'p21bp21cp21dp12ap...",1,1,0,...,1,0,1,1,0,0,1,1,1,0
118163,india,settlement,29,7,26,259383873,"['p1', 'p2p25p8ap8b', 'p21b', 'p8a', 'p8b', 'p...",1,0,0,...,0,0,0,0,0,0,1,1,0,0
118164,in-a-gadda-da-vida,song,8,3,7,203100816,"['p8a', 'p8a', 'p8a', 'p8a', 'p8b', 'p8a', 'p2...",0,0,0,...,0,0,0,0,0,0,1,1,0,0
118165,interfaithfamily,organization,2,2,1,241033740,"['p8a', 'p5']",0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [5]:
# drop potential duplicates
types_one_hot.drop_duplicates(subset=['id'], keep='first',
                             inplace=True)
types_one_hot.shape

(118166, 65)

In [6]:
# drop duplicates from types and subclasses in types_one_hot
subclasses_ids = subclasses_one_hot['id'].to_list()
types_one_hot = types_one_hot[~types_one_hot['id'].isin(subclasses_ids)].reset_index(drop=True)
types_one_hot

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,1988,album,4,2,4,89520215,"['p3a', 'p5', 'p5', 'p3a']",0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,196,year,1,1,1,250210719,['p8b'],0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,53,year,2,1,1,333976287,['p8b'],0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2007,year,104,6,57,179643001,"['p25p8ap8b', 'p8b', 'p8b', 'p25', 'p8b', 'p8b...",1,0,0,...,0,0,0,0,0,0,1,1,0,0
4,2,aircraft,21,4,16,472381603,"['p8b', 'p8b', 'p8b', 'p3a', 'p8b', 'p5', 'p8b...",0,0,0,...,0,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116443,india,city,251,29,206,205155758,"['p5p4p1p15bp23ap24p8ap3a', 'p21bp21cp21dp12ap...",1,1,0,...,1,0,1,1,0,0,1,1,1,0
116444,india,settlement,29,7,26,259383873,"['p1', 'p2p25p8ap8b', 'p21b', 'p8a', 'p8b', 'p...",1,0,0,...,0,0,0,0,0,0,1,1,0,0
116445,in-a-gadda-da-vida,song,8,3,7,203100816,"['p8a', 'p8a', 'p8a', 'p8a', 'p8b', 'p8a', 'p2...",0,0,0,...,0,0,0,0,0,0,1,1,0,0
116446,interfaithfamily,organization,2,2,1,241033740,"['p8a', 'p5']",0,0,0,...,0,0,0,1,0,0,1,0,0,0


### 1.1.3 Get same column names and labels for subclass and types
label subclass = 0
label types = 1
label negatives = 2

In [7]:
# put label = 0 for subclasses
subclasses_one_hot['label'] = [0] * len(subclasses_one_hot)

In [8]:
# put label = 1 for types
types_one_hot['label'] = [1] * len(types_one_hot)

In [9]:
# show column anmes for subclasses
subclasses_columns = subclasses_one_hot.columns.to_list()

In [10]:
# show column anmes for types
types_columns = types_one_hot.columns.to_list()

In [11]:
# get difference between two dataframes
list(set(subclasses_columns) - set(types_columns)) # only first column is named differently

['subClass']

In [12]:
# get difference between two dataframes
list(set(types_columns) - set(subclasses_columns)) # only first column is named differently

['instance', 'p31b']

In [13]:
subclasses_one_hot.columns

Index(['subClass', 'class', 'frequency', 'pidspread', 'pldspread', 'id',
       'pids', 'p1', 'p10', 'p11', 'p12a', 'p12b', 'p12c', 'p13', 'p14',
       'p15a', 'p15b', 'p16', 'p2', 'p20a', 'p20b', 'p20c', 'p20d', 'p21a',
       'p21b', 'p21c', 'p21d', 'p22a', 'p22b', 'p23a', 'p23b', 'p23c', 'p23d',
       'p23e', 'p24', 'p25', 'p26', 'p27a', 'p27b', 'p28a', 'p28b', 'p28c',
       'p28d', 'p29a', 'p29c', 'p30a', 'p30b', 'p31a', 'p34', 'p36', 'p37',
       'p38', 'p39', 'p3a', 'p4', 'p42', 'p43', 'p5', 'p6', 'p7', 'p8a', 'p8b',
       'p8c', 'p8d', 'label'],
      dtype='object')

In [14]:
types_one_hot.columns

Index(['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id',
       'pids', 'p1', 'p10', 'p11', 'p12a', 'p12b', 'p12c', 'p13', 'p14',
       'p15a', 'p15b', 'p16', 'p2', 'p20a', 'p20b', 'p20c', 'p20d', 'p21a',
       'p21b', 'p21c', 'p21d', 'p22a', 'p22b', 'p23a', 'p23b', 'p23c', 'p23d',
       'p23e', 'p24', 'p25', 'p26', 'p27a', 'p27b', 'p28a', 'p28b', 'p28c',
       'p28d', 'p29a', 'p29c', 'p30a', 'p30b', 'p31a', 'p31b', 'p34', 'p36',
       'p37', 'p38', 'p39', 'p3a', 'p4', 'p42', 'p43', 'p5', 'p6', 'p7', 'p8a',
       'p8b', 'p8c', 'p8d', 'label'],
      dtype='object')

In [15]:
# rename first column
subclasses_one_hot.rename(columns={'subClass': 'instance'}, inplace=True)
subclasses_one_hot.head()

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d,label
0,10,decade,2,1,2,108191420,['p3a'],0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,abbess,nun,12,7,11,403873869,"['p4p8ap3a', 'p3a', 'p8b', 'p4', 'p8b', 'p27a'...",1,0,0,...,0,0,0,0,0,1,1,0,0,0
2,abbot,person,48,7,32,156276663,"['p4p1', 'p1', 'p1', 'p1', 'p1', 'p8b', 'p3a',...",1,0,0,...,0,0,1,0,0,1,1,0,0,0
3,actuary,statistician,3,2,3,512259817,['p4p8a'],0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,activist,person,172,16,101,485263422,"['p4p23dp2p39p1p23ap23bp8ap3ap43', 'p6p8bp3ap4...",1,0,0,...,0,1,1,1,0,1,1,0,0,0


In [16]:
# insert column p31b into subclasses
p31b_location = types_one_hot.columns.get_loc('p31b')
p31b_location

48

In [17]:
# insert p31b into subclasses
subclasses_one_hot.insert(loc=p31b_location, column='p31b', value=[0]*len(subclasses_one_hot))

In [18]:
# split subclass to 5% percent for testing
subclasses_one_hot_train, subclasses_one_hot_test = util_preprocessing.get_train_test(subclasses_one_hot)
print('Shape train: {} || Shape test: {}'.format(subclasses_one_hot_train.shape, 
                                                 subclasses_one_hot_test.shape))

Shape train: (3608, 66) || Shape test: (190, 66)


In [30]:
# split types to 5% percent for testing
types_one_hot_train, types_one_hot_test = util_preprocessing.get_train_test(types_one_hot)
print('Shape train: {} || Shape test: {}'.format(types_one_hot_train.shape, 
                                                 types_one_hot_test.shape))

Shape train: (110625, 66) || Shape test: (5823, 66)


In [31]:
# export types & subclass_train
subclasses_one_hot_test.to_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/test/subclass_test.csv',
                              sep=";")
types_one_hot_test.to_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/test/types_test.csv',
                              sep=";")

In [21]:
# get 5 non-overlapping validation sets for subclasses
folder_out_subclasses = '/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/subclass_'
one_fifth_subclasses = int(len(subclasses_one_hot_train)/5)

In [44]:
# get 5 non-overlapping validation sets for subclasses
sanity_check_ids = []
validation = subclasses_one_hot_train.copy()

In [45]:
# get 5 non-overlapping validation sets for subclasses
for i in range(0,5):
    
    # remove the ones which were already drawn
    validation = validation[~validation.id.isin(sanity_check_ids)]
    # print(len(validation))
    
    if i < 4:
        # get random sample from types which equals one fifth of the data
        validation_data = validation.sample(one_fifth_subclasses)
    
        # get unique ids from validation set for sanity check
        sanity_check_ids.extend(list(set(validation_data.id)))
        print(len(list(set(validation_data.id))))
        #print(sanity_check_ids)
    else:
        validation_data = validation

        
    # export validation set
    file_name = folder_out_subclasses+str(i)+'.csv'
    validation_data.to_csv(file_name, sep=";")

721
721
721
721


In [46]:
# get 5 non-overlapping validation sets for types
folder_out_types = '/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/types_'
one_fifth_types = int(len(types_one_hot_train)/5)

In [47]:
# get 5 non-overlapping validation sets for types
sanity_check_ids = []
validation = types_one_hot_train.copy()

In [48]:
# get 5 non-overlapping validation sets for types
for i in range(0,5):
    
    # remove the ones which were already drawn
    validation = validation[~validation.id.isin(sanity_check_ids)]
    # print(len(validation))
    
    if i < 4:
        # get random sample from types which equals one fifth of the data
        validation_data = validation.sample(one_fifth_types)
    
        # get unique ids from validation set for sanity check
        sanity_check_ids.extend(list(set(validation_data.id)))
        print(len(list(set(validation_data.id))))
        #print(sanity_check_ids)
    else:
        validation_data = validation

        
    # export validation set
    file_name = folder_out_types+str(i)+'.csv'
    validation_data.to_csv(file_name, sep=";")

22125
22125
22125
22125


### 1.1.4 Get negatives

In [23]:
# read one hot encoded data
negative_1 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/negatives_1_one_hot_encoded.csv', sep=";",
                         index_col=0)

negative_2 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/negatives_2_one_hot_encoded.csv', sep=";",
                        index_col=0)

negative_3 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/negatives_3_one_hot_encoded.csv', sep=";",
                        index_col=0)

negative_4 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/negatives_4_one_hot_encoded.csv', sep=";",
                        index_col=0)

negative_5 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/negatives_5_one_hot_encoded.csv', sep=";", 
                        index_col=0)

negative_6 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/negatives_6_one_hot_encoded.csv', sep=";",
                        index_col=0)

In [4]:
# stack all test data together 
used_negatives_id = []
negatives_test_all = pd.DataFrame()

for negative in [negative_1, negative_2, negative_3,
                negative_4, negative_5, negative_6]:
    
        negatives_train, negatives_test = util_preprocessing.get_train_test(negative)
        
        # debug
        print(len(negatives_test))
        
        # concat test to negatives_test
        negatives_test_all = pd.concat([negatives_test_all, negatives_test])
        
        # debug
        print(f'Negatives test {len(negatives_test_all)}')

57018
Negatives test 57018
57018
Negatives test 114036
57018
Negatives test 171054
57018
Negatives test 228072
57018
Negatives test 285090
57018
Negatives test 342108


In [6]:
# sanity check
negatives_test_all.head()

,_id,instance,class,frequency,pidspread,pldspread,modifications,p1,p10,p11,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
491202,503817711,archbishop,camp,1,1,1,['p3a'],0,0,0,...,0,0,0,0,0,0,0,0,0,0
562441,472651154,authentication,dinosaur,1,1,1,['p8a'],0,0,0,...,0,0,0,0,0,0,1,0,0,0
206256,36750076,aerosol,topic,10,6,10,"['p5p10', 'p8c', 'p10', 'p4', 'p5', 'p8b', 'p5...",0,1,0,...,1,0,0,1,0,0,0,1,1,0
92205,131897821,m,flute,2,2,2,"['p5', 'p21a']",0,0,0,...,0,0,0,1,0,0,0,0,0,0
583214,158722275,ayano,friend,1,1,1,['p8a'],0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [7]:
# get unique ids in negatives
len(list(set(negatives_test_all['_id'])))

342108

In [ ]:
# rename columns of negatives
negatives_test_all.rename(columns={'_id': 'id',
                                  'modifications': 'pids'}, 
                         inplace=True)

In [11]:
# sanity check
negatives_test_all.head()

,id,instance,class,frequency,pidspread,pldspread,pids,p1,p10,p11,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
491202,503817711,archbishop,camp,1,1,1,['p3a'],0,0,0,...,0,0,0,0,0,0,0,0,0,0
562441,472651154,authentication,dinosaur,1,1,1,['p8a'],0,0,0,...,0,0,0,0,0,0,1,0,0,0
206256,36750076,aerosol,topic,10,6,10,"['p5p10', 'p8c', 'p10', 'p4', 'p5', 'p8b', 'p5...",0,1,0,...,1,0,0,1,0,0,0,1,1,0
92205,131897821,m,flute,2,2,2,"['p5', 'p21a']",0,0,0,...,0,0,0,1,0,0,0,0,0,0
583214,158722275,ayano,friend,1,1,1,['p8a'],0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
# change positions of columns just like in subclasses
ids = negatives_test_all['id']
del negatives_test_all['id']

In [9]:
subclasses_one_hot_test.head()

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d,label
1070,quartermaster,person,5,2,5,53221377,"['p4', 'p8b']",0,0,0,...,0,0,0,0,0,0,1,0,0,0
354,doughnut,food,330,17,271,382483910,"['p5p4p2p15ap1p27ap28bp8cp3a', 'p5p28cp3a', 'p...",1,1,0,...,0,1,1,0,0,1,0,1,1,0
881,mycologist,person,3,3,2,257486612,"['p4', 'p26', 'p5']",0,0,0,...,0,0,1,0,0,0,0,0,0,0
196,blog,website,3892,42,2351,4660184,['p7p6p5p4p2p1p43p26p15ap25p15bp24p20cp20bp20d...,1,1,0,...,0,1,1,1,1,1,1,1,0,0
3518,stupa,place,44,9,40,374670915,"['p21dp8ap8c', 'p5p8a', 'p8a', 'p20b', 'p8a', ...",1,0,0,...,0,0,1,0,0,1,1,1,0,0


In [16]:
# insert id before pids
negatives_test_all.insert(loc=5, value=ids, column='id')
negatives_test_all.head()

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
491202,archbishop,camp,1,1,1,503817711,['p3a'],0,0,0,...,0,0,0,0,0,0,0,0,0,0
562441,authentication,dinosaur,1,1,1,472651154,['p8a'],0,0,0,...,0,0,0,0,0,0,1,0,0,0
206256,aerosol,topic,10,6,10,36750076,"['p5p10', 'p8c', 'p10', 'p4', 'p5', 'p8b', 'p5...",0,1,0,...,1,0,0,1,0,0,0,1,1,0
92205,m,flute,2,2,2,131897821,"['p5', 'p21a']",0,0,0,...,0,0,0,1,0,0,0,0,0,0
583214,ayano,friend,1,1,1,158722275,['p8a'],0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [17]:
# get label as last column
negatives_test_all['label'] = [2] * len(negatives_test_all)
negatives_test_all.shape

(342108, 66)

In [18]:
# export for future uses
negatives_test_all.to_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/test/negatives_test.csv',
                            sep=";")

In [4]:
# export for future uses
negatives_test_all = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/test/negatives_test.csv',
                            sep=";", index_col=0)

In [20]:
# stack negatives with subclasses and types together
test_set = pd.concat([negatives_test_all, types_one_hot_test, subclasses_one_hot_test])
test_set = test_set.sample(frac=1).reset_index(drop=True) # shuffle data
test_set.shape

(348121, 66)

In [23]:
test_set.head()

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d,label
0,pipkin,comedian,1,1,1,106178699,['p8a'],0,0,0,...,0,0,0,0,0,1,0,0,0,2
1,animal,herbicide,4,4,4,422693865,"['p3a', 'p21b', 'p23a', 'p23d']",0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,crs,index,2,2,2,470578787,"['p5', 'p21a']",0,0,0,...,0,0,1,0,0,0,0,0,0,2
3,oldham,wagon,1,1,1,117505605,['p3a'],0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,tyack,woman,1,1,1,372230967,['p12a'],0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [27]:
test_set.to_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/test/test_all.csv',
                sep=";")

# 1.1.5 Get validation set for negatives

In [5]:
# get ids of negatives test to exclude from analysis
negatives_test_ids = negatives_test_all.id.to_list()

In [6]:
# get validation set
validation_1 = pd.DataFrame()
validation_2 = pd.DataFrame()
validation_3 = pd.DataFrame()
validation_4 = pd.DataFrame()
validation_5 = pd.DataFrame()

for negative in [negative_1, negative_2, negative_3,
                negative_4, negative_5, negative_6]:
    
    
    # delete all instances without test instances
    negative = negative[~negative['_id'].isin(negatives_test_ids)]
    
    # get ids for new place
    ids = negative['_id']
    del negative['_id']
    
    # insert ids in negatives
    negative.insert(loc=5, value=ids, column='id')
    
    # rename negatives['modifications']
    negative.rename(columns={'modifications': 'pids'}, inplace=True)
    negative['label'] = [2] * len(negative)
    
    # split into 5 equally sized chunks for validaiton
    negative = np.array_split(negative, 5)
    
    validation_1 = pd.concat([validation_1, negative[0]])
    validation_2 = pd.concat([validation_2, negative[1]])
    validation_3 = pd.concat([validation_3, negative[2]])
    validation_4 = pd.concat([validation_4, negative[3]])
    validation_5 = pd.concat([validation_5, negative[4]])
    
    print(len(validation_1))
    print('-------------')

/home/linda/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


216669
-------------
433338
-------------
650007
-------------
866676
-------------
1083345
-------------
1300014
-------------


In [9]:
# get validation set
validation_1 = pd.DataFrame()
validation_2 = pd.DataFrame()
validation_3 = pd.DataFrame()
validation_4 = pd.DataFrame()
validation_5 = pd.DataFrame()

for negative in [negative_1, negative_2, negative_3,
                negative_4, negative_5, negative_6]:
    
    
    # delete all instances without test instances
    negative = negative[~negative['_id'].isin(negatives_test_ids)]
    
    # get ids for new place
    ids = negative['_id']
    del negative['_id']
    
    # insert ids in negatives
    negative.insert(loc=5, value=ids, column='id')
    
    # rename negatives['modifications']
    negative.rename(columns={'modifications': 'pids'}, inplace=True)
    negative['label'] = [2] * len(negative)
    
    # split into 5 equally sized chunks for validaiton
    negative = np.array_split(negative, 5)
    
    validation_1 = pd.concat([validation_1, negative[0]])
    validation_2 = pd.concat([validation_2, negative[1]])
    validation_3 = pd.concat([validation_3, negative[2]])
    validation_4 = pd.concat([validation_4, negative[3]])
    validation_5 = pd.concat([validation_5, negative[4]])
    
    print(len(validation_1))
    print('-------------')

/home/linda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


216669
-------------
433338
-------------
650007
-------------
866676
-------------
1083345
-------------
1300014
-------------


In [10]:
# export validation data for negatives
validation_1.to_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/negatives_0.csv', sep=";")
validation_2.to_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/negatives_1.csv', sep=";")
validation_3.to_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/negatives_2.csv', sep=";")
validation_4.to_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/negatives_3.csv', sep=";")
validation_5.to_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/negatives_4.csv', sep=";")

In [11]:
validation_1.head()

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d,label
0,1866,railway,1,1,1,1643854,['p8b'],0,0,0,...,0,0,0,0,0,0,1,0,0,2
1,1960,library,6,5,6,1648888,"['p23a', 'p12a', 'p20a', 'p8a', 'p8b', 'p8a']",0,0,0,...,0,0,0,0,0,1,1,0,0,2
2,1960,costume,3,1,3,1673500,['p2'],0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,1975,epithet,1,1,1,1678428,['p5'],0,0,0,...,0,0,1,0,0,0,0,0,0,2
4,2000,habitat,1,1,1,1678435,['p8b'],0,0,0,...,0,0,0,0,0,0,1,0,0,2


### 1.1.6 Get validation sets

In [3]:
# get validation sets
for i in range(0,5):
    
    # read data
    negative = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/negatives_' +str(i)+'.csv', sep=";",
                          index_col=0)
    subclass = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/subclass_' +str(i)+'.csv', sep=";",
                          index_col=0)
    types = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/types_' +str(i)+'.csv', sep=";",
                          index_col=0)
    
    # stack data together and shuffle
    validation = pd.concat([negative, subclass, types]).sample(frac=1).reset_index(drop=True)
    
    # debug
    print(validation.shape)
    
    # export
    validation.to_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/validation_' +str(i)+'.csv',
                     sep=";")

(1322860, 66)
(1322860, 66)
(1322854, 66)
(1322854, 66)
(1322857, 66)


### 1.1.7 Downsample to subclass for 4 training folds

In [15]:
# validation folder
validation_folder = '/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/validation_'

In [16]:
training_folder = '/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/train/downsampled_to_subclass/train_'

In [17]:
# loop through all folds
for i in range(0,5):
    # all folds
    folds = [0,1,2,3,4]
    
    # remove fold
    folds.remove(i)
    
    data = pd.DataFrame()
    
    # get all rest of validation fold and downsample to types
    for j in folds:
        validation_file_name = validation_folder + str(j) + '.csv'
        
        # get validation file
        validation_file = pd.read_csv(validation_file_name, sep=";", index_col=0)
        
        # include all subclasses to data
        subclasses = validation_file[validation_file['label'] == 0]
        
        # types
        types = validation_file[validation_file['label'] == 1]
        types = types.sample(len(subclasses))
        
        # negatives
        negatives = validation_file[validation_file['label'] == 2]
        negatives = negatives.sample(len(subclasses))
        
        data = pd.concat([data, subclasses, types, negatives]).reset_index(drop=True)
        data = data.sample(frac=1).reset_index(drop=True)
    
    print(len(data))
    print(data.label.value_counts())
    # export data
    data_out = training_folder + str(i) + '.csv'
    data.to_csv(data_out, sep=";")
        

/home/linda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


8661
2    2887
1    2887
0    2887
Name: label, dtype: int64
8661
2    2887
1    2887
0    2887
Name: label, dtype: int64
8661
2    2887
1    2887
0    2887
Name: label, dtype: int64
8661
2    2887
1    2887
0    2887
Name: label, dtype: int64
8652
2    2884
1    2884
0    2884
Name: label, dtype: int64


### 1.1.8 Downsample to subclass  and upsample to types for 4 training folds

In [19]:
training_folder = '/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/train/downsampled_to_types/train_'

In [20]:
# loop through all folds
for i in range(0,5):
    # all folds
    folds = [0,1,2,3,4]
    
    # remove fold
    folds.remove(i)
    
    data = pd.DataFrame()
    
    # get all rest of validation fold and downsample to types
    for j in folds:
        validation_file_name = validation_folder + str(j) + '.csv'
        
        # get validation file
        validation_file = pd.read_csv(validation_file_name, sep=";", index_col=0)
        
        # include all subclasses to data
        subclasses = validation_file[validation_file['label'] == 0]
        
        # types
        types = validation_file[validation_file['label'] == 1]
        
        # negatives
        negatives = validation_file[validation_file['label'] == 2]
        negatives = negatives.sample(len(types))
        
        # copy x times subclasses
        copy_times = math.floor(len(types)/len(subclasses))
        subclasses_all = pd.concat([subclasses]*copy_times, ignore_index=True)
 
        # rest of subclasses
        rest = len(types) - len(subclasses_all)
        subclasses = subclasses.sample(rest)
        subclasses_all = pd.concat([subclasses_all, subclasses], ignore_index=True)
        
        #data = pd.concat([data, subclasses, types, negatives]).reset_index(drop=True)
        data = pd.concat([data, types, negatives, subclasses_all]).reset_index(drop=True)
        data = data.sample(frac=1).reset_index(drop=True)
    
    # debugging
    print(len(data))
    print(data.label.value_counts())
    
    # export data
    data_out = training_folder + str(i) + '.csv'
    data.to_csv(data_out, sep=";")
        

265500
2    88500
1    88500
0    88500
Name: label, dtype: int64
265500
2    88500
1    88500
0    88500
Name: label, dtype: int64
265500
2    88500
1    88500
0    88500
Name: label, dtype: int64
265500
2    88500
1    88500
0    88500
Name: label, dtype: int64
265500
2    88500
1    88500
0    88500
Name: label, dtype: int64


# 2 Count Based

## 2.1 Subclasses

In [24]:
# read count based subclasses
subclasses = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/subclasses_count_based.csv', sep=";",
                        index_col=0)
subclasses

,subClass,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,10,decade,2,1,2,108191420,['p3a'],0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,abbess,nun,12,7,11,403873869,"['p4p8ap3a', 'p3a', 'p8b', 'p4', 'p8b', 'p27a'...",1,0,0,...,2,0,0,0,0,0,1,2,0,0
2,abbot,person,48,7,32,156276663,"['p4p1', 'p1', 'p1', 'p1', 'p1', 'p8b', 'p3a',...",13,0,0,...,1,0,0,2,0,0,6,6,0,0
3,actuary,statistician,3,2,3,512259817,['p4p8a'],0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,activist,person,172,16,101,485263422,"['p4p23dp2p39p1p23ap23bp8ap3ap43', 'p6p8bp3ap4...",15,0,0,...,5,0,8,1,1,0,17,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793,zimbabweans,person,1,1,1,144540438,['p1'],1,0,0,...,0,0,0,0,0,0,0,0,0,0
3794,zine,work,32,10,26,170394103,"['p5p1p20a', 'p8a', 'p8a', 'p8a', 'p3a', 'p3a'...",6,0,0,...,0,0,0,2,0,0,10,1,0,0
3795,zine,periodical,23,5,15,189735779,"['p4p1', 'p1p3a', 'p3a', 'p12a', 'p3a', 'p1', ...",6,0,0,...,2,0,0,0,0,0,2,0,0,0
3796,zine,publication,124,12,96,418332088,"['p5p34p4p1p8ap28dp3a', 'p1p8a', 'p1', 'p1', '...",25,0,0,...,2,0,0,8,0,0,36,0,0,0


In [25]:
# rename subClass column
subclasses.rename(columns={'subClass': 'instance'},
                 inplace=True)

In [26]:
# get label for subclasses
subclasses['label'] = [0] * len(subclasses)

## 2.2 Types

In [27]:
# read types
types = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/types_count_based.csv', 
                    sep=";", index_col=0)
types

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,1988,album,4,2,4,89520215,"['p3a', 'p5', 'p5', 'p3a']",0,0,0,...,0,0,0,2,0,0,0,0,0,0
1,196,year,1,1,1,250210719,['p8b'],0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,53,year,2,1,1,333976287,['p8b'],0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2007,year,104,6,57,179643001,"['p25p8ap8b', 'p8b', 'p8b', 'p25', 'p8b', 'p8b...",1,0,0,...,0,0,0,0,0,0,6,74,0,0
4,2,aircraft,21,4,16,472381603,"['p8b', 'p8b', 'p8b', 'p3a', 'p8b', 'p5', 'p8b...",0,0,0,...,0,0,0,1,0,0,3,7,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118162,india,city,251,29,206,205155758,"['p5p4p1p15bp23ap24p8ap3a', 'p21bp21cp21dp12ap...",22,1,0,...,2,0,2,19,0,0,35,3,1,0
118163,india,settlement,29,7,26,259383873,"['p1', 'p2p25p8ap8b', 'p21b', 'p8a', 'p8b', 'p...",1,0,0,...,0,0,0,0,0,0,5,2,0,0
118164,in-a-gadda-da-vida,song,8,3,7,203100816,"['p8a', 'p8a', 'p8a', 'p8a', 'p8b', 'p8a', 'p2...",0,0,0,...,0,0,0,0,0,0,5,1,0,0
118165,interfaithfamily,organization,2,2,1,241033740,"['p8a', 'p5']",0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [28]:
# get label for types
types['label'] = [1] * len(types)

In [29]:
# get difference between types and subclasses
list(set(types.columns.to_list()) - set(subclasses.columns.to_list()))

['p31b']

In [30]:
# get location of that column
types.columns.get_loc('p31b')

48

In [31]:
# column into subclasses
subclasses.insert(loc=48,value=[0]*len(subclasses), column='p31b')

In [32]:
subclasses.shape

(3798, 66)

In [33]:
# drop duplicates in types
types = types.drop_duplicates(subset=['id'])
types

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d,label
0,1988,album,4,2,4,89520215,"['p3a', 'p5', 'p5', 'p3a']",0,0,0,...,0,0,2,0,0,0,0,0,0,1
1,196,year,1,1,1,250210719,['p8b'],0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,53,year,2,1,1,333976287,['p8b'],0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,2007,year,104,6,57,179643001,"['p25p8ap8b', 'p8b', 'p8b', 'p25', 'p8b', 'p8b...",1,0,0,...,0,0,0,0,0,6,74,0,0,1
4,2,aircraft,21,4,16,472381603,"['p8b', 'p8b', 'p8b', 'p3a', 'p8b', 'p5', 'p8b...",0,0,0,...,0,0,1,0,0,3,7,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118162,india,city,251,29,206,205155758,"['p5p4p1p15bp23ap24p8ap3a', 'p21bp21cp21dp12ap...",22,1,0,...,0,2,19,0,0,35,3,1,0,1
118163,india,settlement,29,7,26,259383873,"['p1', 'p2p25p8ap8b', 'p21b', 'p8a', 'p8b', 'p...",1,0,0,...,0,0,0,0,0,5,2,0,0,1
118164,in-a-gadda-da-vida,song,8,3,7,203100816,"['p8a', 'p8a', 'p8a', 'p8a', 'p8b', 'p8a', 'p2...",0,0,0,...,0,0,0,0,0,5,1,0,0,1
118165,interfaithfamily,organization,2,2,1,241033740,"['p8a', 'p5']",0,0,0,...,0,0,1,0,0,1,0,0,0,1


In [59]:
types.id.value_counts()

137103359    1
75160176     1
79620726     1
134931063    1
315529848    1
            ..
62510959     1
438124402    1
190383989    1
307304311    1
380223453    1
Name: id, Length: 118166, dtype: int64

## 2.3 Get test for types and subclasses

In [11]:
# subclasses
subclasses_one_hot = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/test/subclass_test.csv',
                                sep=";", index_col=0)
subclasses_one_hot = subclasses_one_hot.id.to_list()
subclasses_test = subclasses[subclasses['id'].isin(subclasses_one_hot)]

In [12]:
# types
types_one_hot = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/test/types_test.csv',
                                sep=";", index_col=0)
types_one_hot = types_one_hot.id.to_list()
types_test = types[types['id'].isin(types_one_hot)]

## 2.4 Negatives test

In [44]:
# read one hot encoded data
negative_1 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/negatives_1_count_based.csv', sep=";",
                        index_col=0)

negative_2 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/negatives_2_count_based.csv', sep=";",
                        index_col=0)

negative_3 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/negatives_3_count_based.csv', sep=";", 
                         index_col=0)

negative_4 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/negatives_4_count_based.csv', sep=";",
                        index_col=0)

negative_5 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/negatives_5_count_based.csv', sep=";",
                        index_col=0)

negative_6 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/negatives_6_count_based.csv', sep=";",
                        index_col=0)

negative_7 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/negatives_7_count_based.csv', sep=";",
                        index_col=0)

negative_8 = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/negatives_8_count_based.csv', sep=";",
                        index_col=0)

In [18]:
negative_1.head()

,_id,instance,class,frequency,pidspread,pldspread,modifications,p1,p10,p11,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,1643854,1866,railway,1,1,1,['p8b'],0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1648888,1960,library,6,5,6,"['p23a', 'p12a', 'p20a', 'p8a', 'p8b', 'p8a']",0,0,0,...,0,0,0,0,0,0,2,1,0,0
2,1673500,1960,costume,3,1,3,['p2'],0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1678428,1975,epithet,1,1,1,['p5'],0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1678435,2000,habitat,1,1,1,['p8b'],0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [19]:
# get ids of negatives
negatives_id = pd.read_csv('/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/test/negatives_test.csv',
                                sep=";", index_col=0)
negatives_id = negatives_id.id.to_list()

In [22]:
# loop through all negatives to get count test set
negative_test = pd.DataFrame()
for negative in [negative_1, negative_2, negative_3, negative_4,
                negative_5, negative_6, negative_7, negative_8]:
    
    # subset data
    negative = negative[negative['_id'].isin(negatives_id)]
    
    # get id column
    ids = negative['_id']
    del negative['_id']
    
    # rename columns
    negative.rename(columns={'modifications': 'pids'}, 
                   inplace=True)
    
    # insert ids into negatives
    negative.insert(loc=5, column='id', value = ids)
    
    # get label
    negative['label'] = [2] * len(negative)
    
    # concat to test set for negatives
    negative_test = pd.concat([negative_test, negative],
                             ignore_index=True)

/home/linda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
# test set complete
test_count = pd.concat([negative_test, types_test, 
                       subclasses_test], ignore_index=True)
test_count = test_count.sample(frac=1)
test_count.label.value_counts()

2    342108
1      5823
0       190
Name: label, dtype: int64

In [25]:
# sanity check
test_count.shape

(348121, 66)

In [26]:
# export data
test_count.to_csv('/path/to/9_FINAL/data/machine_learning/count_based/three_class/test.csv',sep=";")

## 2.5 validation sets

In [11]:
# get validation folder
validation_folder = '/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/validation/validation_'

In [14]:
# loop through all negatives to get count validation set

for i in range(1,5):
    validation_file_name = validation_folder + str(i) + '.csv'
    validation_file = pd.read_csv(validation_file_name, sep=";",
                                 index_col=0)
    
    # get indices of validation file
    validation_file = validation_file.id.to_list()
    
    # get subsets of subclasses and types
    subclasses_validation = subclasses[subclasses['id'].isin(validation_file)]
    types_validation = types[types['id'].isin(validation_file)]
    
    # initialize negatives
    negatives = pd.DataFrame()

    for negative in [negative_1, negative_2, negative_3, negative_4,
                negative_5, negative_6, negative_7, negative_8]:
    
        # subset data
        negative = negative[negative['_id'].isin(validation_file)]
    
        # get id column
        ids = negative['_id']
        del negative['_id']
    
        # rename columns
        negative.rename(columns={'modifications': 'pids'}, 
                   inplace=True)
    
        # insert ids into negatives
        negative.insert(loc=5, column='id', value = ids)
    
        # get label
        negative['label'] = [2] * len(negative)
    
        # concat to test set for negatives
        negatives = pd.concat([negatives, negative],
                             ignore_index=True)
    
    validation = pd.concat([negatives, types_validation, 
                            subclasses_validation],
                          ignore_index=True)
    # shuffle data
    validation = validation.sample(frac=1)
    
    # sanity check
    print(validation.label.value_counts())
    
    # export data
    out_data = '/path/to/9_FINAL/data/machine_learning/count_based/three_class/validation/validation_'+str(i)+'.csv'
    validation.to_csv(out_data, sep=";")

/home/linda/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/home/linda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2    1300014
1      22435
0        721
Name: label, dtype: int64
2    1300008
1      22444
0        721
Name: label, dtype: int64
2    1300008
1      22471
0        721
Name: label, dtype: int64
2.0    1300008
1.0      22447
0.0        724
Name: label, dtype: int64


## 2.6 train sets downsampled to subclass

In [51]:
# get train folder
train_folder = '/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/train/downsampled_to_subclass/train_'

In [ ]:
# loop through all negatives to get count train set

for i in range(0,5):
    train_file_name = train_folder + str(i) + '.csv'
    train_file = pd.read_csv(train_file_name, sep=";",
                                 index_col=0)
    
    # get indices of train file
    train_ids = train_file.id.to_list()
    types_ids = train_file[train_file['label'] == 1]
    types_ids = types_ids.id.to_list()
    print(len(train_ids))
    
    #print(train_file)
    
    # get subsets of subclasses and types
    subclasses_train = subclasses[subclasses['id'].isin(train_ids)]
    types_train = types[types.id.isin(types_ids)]
    
    # print(types_train)
    
    # initialize negatives
    negatives = pd.DataFrame()

    for negative in [negative_1, negative_2, negative_3, negative_4,
                negative_5, negative_6, negative_7, negative_8]:
    
        # subset data
        negative = negative[negative['_id'].isin(train_ids)]
    
        # get id column
        ids = negative['_id']
        del negative['_id']
    
        # rename columns
        negative.rename(columns={'modifications': 'pids'}, 
                   inplace=True)
    
        # insert ids into negatives
        negative.insert(loc=5, column='id', value = ids)
    
        # get label
        negative['label'] = [2] * len(negative)
    
        # concat to test set for negatives
        negatives = pd.concat([negatives, negative],
                             ignore_index=True)
    
    train = pd.concat([negatives, types_train, 
                            subclasses_train])
    # shuffle data
    train = train.sample(frac=1)
    
    # sanity check
    print(train.label.value_counts())
    
    # export data
    out_data = '/path/to/9_FINAL/data/machine_learning/count_based/three_class/train/downsampled_to_subclass/train_'+str(i)+'.csv'
    train.to_csv(out_data, sep=";")

## 2.7 train sets downsampled to types

In [34]:
# get train folder
train_folder = '/path/to/9_FINAL/data/machine_learning/one_hot_encoded/three_class/train/downsampled_to_types/train_'

In [39]:
# remove unnecessary columns from subclass
subclasses.drop(columns=['instance', 'class', 'frequency',
                        'pidspread', 'pldspread', 'pids'], inplace=True)
subclasses.head()

,id,p1,p10,p11,p12a,p12b,p12c,p13,p14,p15a,...,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d,label
0,108191420,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,403873869,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,2,0,0,0
2,156276663,13,0,0,0,0,0,0,0,0,...,0,0,2,0,0,6,6,0,0,0
3,512259817,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,485263422,15,0,0,2,0,0,0,0,0,...,0,8,1,1,0,17,5,0,0,0


In [45]:
# loop through all negatives to get count train set

for i in range(0,5):
    train_file_name = train_folder + str(i) + '.csv'
    train_file = pd.read_csv(train_file_name, sep=";",
                                 index_col=0)
    
    # get subclass
    subtrain_class = train_file[train_file['label'] == 0].reset_index(drop=True)
    subtrain_class = subtrain_class[['instance', 'class', 'frequency', 'pidspread', 'pldspread', 'id',
                                    'pids']]
    
    
    # get indices of train file
    negative_ids = train_file[train_file['label'] == 2]
    negative_ids = negative_ids.id.to_list()
    types_ids = train_file[train_file['label'] == 1]
    types_ids = types_ids.id.to_list()
    
    #print(train_file)
    
    # get subsets of subclasses and types
    subclasses_train = pd.merge(subtrain_class, subclasses, how='left')
    types_train = types[types.id.isin(types_ids)]
    
    # print(types_train)
    
    # initialize negatives
    negatives = pd.DataFrame()

    for negative in [negative_1, negative_2, negative_3, negative_4,
                negative_5, negative_6, negative_7, negative_8]:
    
        # subset data
        negative = negative[negative['_id'].isin(negative_ids)]
    
        # get id column
        ids = negative['_id']
        del negative['_id']
    
        # rename columns
        negative.rename(columns={'modifications': 'pids'}, 
                   inplace=True)
    
        # insert ids into negatives
        negative.insert(loc=5, column='id', value = ids)
    
        # get label
        negative['label'] = [2] * len(negative)
    
        # concat to test set for negatives
        negatives = pd.concat([negatives, negative],
                             ignore_index=True)
    
    train = pd.concat([negatives, types_train, 
                            subclasses_train])
    # shuffle data
    train = train.sample(frac=1)
    
    # sanity check
    print(train.label.value_counts())
    
    # export data
    out_data = '/path/to/9_FINAL/data/machine_learning/count_based/three_class/train/downsampled_to_types/train_'+str(i)+'.csv'
    train.to_csv(out_data, sep=";")

/home/linda/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
/home/linda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2    88500
1    88500
0    88500
Name: label, dtype: int64
2    88500
1    88500
0    88500
Name: label, dtype: int64
2    88500
1    88500
0    88500
Name: label, dtype: int64
2    88500
1    88500
0    88500
Name: label, dtype: int64
2    88500
1    88500
0    88500
Name: label, dtype: int64


In [47]:
# sanity check
one_fold = pd.read_csv('/path/to/9_FINAL/data/machine_learning/count_based/three_class/train/downsampled_to_types/train_0.csv',
                      sep=";", index_col=0)
one_fold.head()

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d,label
64921,aids,song,4,2,3,171427810,"['p1', 'p8a']",1,0,0,...,0,0,0,0,0,1,0,0,0,0
106321,procession,holiday,9,6,8,219032376,"['p5', 'p1', 'p4', 'p3a', 'p8b', 'p5', 'p8a']",1,0,0,...,0,0,2,0,0,1,1,0,0,1
57849,patty,blog,3,3,2,202146494,"['p5', 'p8c', 'p25']",0,0,0,...,0,0,1,0,0,0,0,1,0,2
105473,zug,city,9,3,9,248648902,"['p8a', 'p8a', 'p8a', 'p8a', 'p1', 'p5']",1,0,0,...,0,0,1,0,0,4,0,0,0,1
75873,oregano,flora,1,1,1,34540524,['p3a'],0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [48]:
# sanity check
one_fold[one_fold['label'] == 2]

,instance,class,frequency,pidspread,pldspread,id,pids,p1,p10,p11,...,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d,label
57849,patty,blog,3,3,2,202146494,"['p5', 'p8c', 'p25']",0,0,0,...,0,0,1,0,0,0,0,1,0,2
37737,insomnia,window,2,2,2,478684363,"['p8a', 'p2']",0,0,0,...,0,0,0,0,0,1,0,0,0,2
67862,sarah,symptom,2,2,2,225404815,"['p30a', 'p28b']",0,0,0,...,0,0,0,0,0,0,0,0,0,2
3614,army,riot,10,5,10,178211696,"['p8b', 'p8a', 'p43', 'p8a', 'p8b', 'p43', 'p5...",0,0,0,...,0,2,1,0,0,3,2,0,0,2
29557,greed,tribute,1,1,1,223566819,['p8a'],0,0,0,...,0,0,0,0,0,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8132,bead,statue,1,1,1,301115848,['p43'],0,0,0,...,0,1,0,0,0,0,0,0,0,2
28582,foundation,pamphlet,3,2,3,283637897,"['p23a', 'p8a']",0,0,0,...,0,0,0,0,0,1,0,0,0,2
47757,memory,galaxy,1,1,1,386832321,['p8a'],0,0,0,...,0,0,0,0,0,1,0,0,0,2
60037,prin,list,1,1,1,175495625,['p3a'],0,0,0,...,0,0,0,0,0,0,0,0,0,2
